## Thư viện

### Cài thư viện

### Import thư viện

In [ ]:
import re

## Dữ liệu

### Tải dữ liệu

In [ ]:
!wget -O "/content/data.zip" "https://drive.google.com/uc?export=download&id=1lXXGI4aY1NVzwvz3NLHFAvhwk9LBnDMB" 

--2023-04-12 14:12:13--  https://drive.google.com/uc?export=download&id=1lXXGI4aY1NVzwvz3NLHFAvhwk9LBnDMB
Resolving drive.google.com (drive.google.com)... 172.217.218.102, 172.217.218.139, 172.217.218.138, ...
Connecting to drive.google.com (drive.google.com)|172.217.218.102|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-14-3s-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/7iucgq0518tdoq8vk56jk6n5ngcg94cu/1681308675000/02702851125710553677/*/1lXXGI4aY1NVzwvz3NLHFAvhwk9LBnDMB?e=download&uuid=9e132c33-9ad5-47ab-825e-fd231c698d9a [following]
--2023-04-12 14:12:15--  https://doc-14-3s-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/7iucgq0518tdoq8vk56jk6n5ngcg94cu/1681308675000/02702851125710553677/*/1lXXGI4aY1NVzwvz3NLHFAvhwk9LBnDMB?e=download&uuid=9e132c33-9ad5-47ab-825e-fd231c698d9a
Resolving doc-14-3s-docs.googleusercontent.com (doc-14-3s-docs.googleusercontent.com)... 142.251.18.132, 2

In [ ]:
!unzip "data.zip"

Archive:  data.zip
replace SA/Guidelines-SA-Hotel (4-3-2018).pdf? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [ ]:
!unzip "/content/SA/VLSP2018-SA-train-dev-test.zip"

Archive:  /content/SA/VLSP2018-SA-train-dev-test.zip
replace VLSP2018-SA-train-dev-test/3-VLSP2018-SA-Restaurant-test (8-3-2018).txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


### Xử lí dữ liệu

In [ ]:
VNalpha = 'àáãạảăắằẳẵặâấầẩẫậèéẹẻẽêềếểễệđìíĩỉịòóõọỏôốồổỗộơớờởỡợùúũụủưứừửữựỳỵỷỹýÀÁÃẠẢĂẮẰẲẴẶÂẤẦẨẪẬÈÉẸẺẼÊỀẾỂỄỆĐÌÍĨỈỊÒÓÕỌỎÔỐỒỔỖỘƠỚỜỞỠỢÙÚŨỤỦƯỨỪỬỮỰỲỴỶỸÝ'

In [ ]:
def preprocess_txt(file_link):
    '''
    * Xử lí file txt chứa đường dẫn file txt thành một mảng chứa các điểm dữ liệu.
    * Một điểm dữ liệu là tuple (x, y) với:  
      + x: Mảng chứa các từ, một dấu câu xem là một tiếng (giữ dấu câu để xét trong việc dùng n-grams về sau).
      + y: Mảng các tuple (y1, y2). y1 là Aspect, y2 là Sentiment.
    '''
    '''
    Dữ liệu trong file txt sẽ lặp lại theo từng 4 dòng với kết cấu sau:
    #1                                               -> chỉ số
    Khách sạn đẹp                                    -> câu
    {Aspect1, Sentiment1}, {Aspect2, Sentiment2} ... -> aspect và sentiment
                                                     -> dòng trống  
    '''
    with open(file_link, encoding='utf8') as f:
        lines = f.readlines()
        result = []
        temp_str = ""
        for line in lines:
            if line == "" or line[0] == "#":  # Không quan tâm dòng trống hay dòng chỉ số
                continue
            if line[0] == "{":                # Dòng aspect và sentiment
                line = re.findall(r'{[^{^}]+}', line)
                line = [tuple(txt[1:-1].replace(' ', '').split(',')) for txt in line]
                result.append((temp_str, line))
            if line[0] != "{":                # Dòng text
                temp_str = str(line).rstrip()
    return result
            

In [ ]:
preprocess_txt('/content/VLSP2018-SA-train-dev-test/1-VLSP2018-SA-Hotel-train (7-3-2018).txt')[:3]

[('Rộng rãi KS mới nhưng rất vắng. Các dịch vụ chất lượng chưa cao và thiếu.',
  [('HOTEL#DESIGN&FEATURES', 'positive'), ('HOTEL#GENERAL', 'negative')]),
 ('Địa điểm thuận tiện, trong vòng bán kính 1,5km nhiều quán ăn ngon',
  [('LOCATION#GENERAL', 'positive')]),
 ('Phục vụ, view đẹp, vị trí',
  [('SERVICE#GENERAL', 'positive'),
   ('HOTEL#GENERAL', 'positive'),
   ('LOCATION#GENERAL', 'positive')])]

In [ ]:
def get_aspect_list(data):
    '''
    Lấy danh sách aspect từ dữ liệu.
    '''
    result = []
    for text, label_lst in data:
        for asp, sent in label_lst:
            result.append(asp)
    return list(set(result))
def get_sentiment_list(data):
    '''
    Lấy danh sách sentiment từ dữ liệu.
    '''
    result = []
    for text, label_lst in data:
        for asp, sent in label_lst:
            result.append(sent)
    return list(set(result))
def process_label(data):
    '''
    Gom nhóm dữ liệu theo label.
    Đầu vào: Dữ liệu theo chuẩn đầu ra của hàm preprocess_txt.
    Đầu ra: Dữ liệu kiểu dict:
      + key là aspect.
      + value là một mảng các tuple (x, y). x là text, y là sentiment.
    sentiment được đánh số:
      1: negative
      2: positive
      3: neutral
      0: not mentioned
    '''
    sent_list = get_sentiment_list(data)
    asp_list = get_aspect_list(data)
    asp_map = {'negative' : 1, 'positive' : 2, 'neutral' : 3}
    result = {}
    for asp in asp_list:
        result[asp] = {}
    for text, label_lst in data:
        for asp in asp_list:
            result[asp][text] = 0
        for asp, sent in label_lst:
            result[asp][text] = asp_map[sent]
    return result

In [ ]:
def process_raw_dataset(text):
    '''
    Xử lí các vấn đề sau:
    - Viết hoa thường lẫn lộn
    - Viết dấu, không dấu lẫn lộn
    - Xoá số, dấu câu, kí tự đặc biệt
    '''
    text = text.lower()
    VNalpha = 'àáãạảăắằẳẵặâấầẩẫậèéẹẻẽêềếểễệđìíĩỉịòóõọỏôốồổỗộơớờởỡợùúũụủưứừửữựỳỵỷỹýÀÁÃẠẢĂẮẰẲẴẶÂẤẦẨẪẬÈÉẸẺẼÊỀẾỂỄỆĐÌÍĨỈỊÒÓÕỌỎÔỐỒỔỖỘƠỚỜỞỠỢÙÚŨỤỦƯỨỪỬỮỰỲỴỶỸÝ'
    VNkodau = 'aaaaaaaaaaaaaaaaaeeeeeeeeeeediiiiiooooooooooooooooouuuuuuuuuuuyyyyyaaaaaaaaaaaaaaaaaeeeeeeeeeeediiiiiooooooooooooooooouuuuuuuuuuuyyyyy'
    for idx in range(len(VNalpha)):
        text = text.replace(VNalpha[idx], VNkodau[idx])
    text = re.sub(r'[^(a-zA-Z )]+', '', text)
    return text

In [ ]:
process_raw_dataset("Hôm nay, tôi ĐI HỌC!")

'hom nay toi di hoc'

## Mô hình

### Hotel

#### Huấn luyện mô hình

In [ ]:
# Xử lí dữ liệu theo các hàm phía trên
processed_txt_data = preprocess_txt('/content/VLSP2018-SA-train-dev-test/1-VLSP2018-SA-Hotel-train (7-3-2018).txt')
processed_txt_data = [(process_raw_dataset(text), label) for text, label in processed_txt_data]
data = process_label(processed_txt_data)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
import pandas as pd

In [ ]:
# dựng corpus (chứa tất cả các text), dựng mô hình để tạo BoW
corpus = [text for txt in list(data.values()) for text in list(txt.keys())]
vectorizer = CountVectorizer()
vectorizer.fit_transform(corpus)

<97119x2775 sparse matrix of type '<class 'numpy.int64'>'
	with 4219347 stored elements in Compressed Sparse Row format>

In [ ]:
# Tạo mô hình
# Mỗi aspect có 1 mô hình riêng, đầu vào là text, đầu ra là 0, 1, 2, hoăc 3 tương ứng với not mentioned, negative, positive, neutral
asp_lst = get_aspect_list(processed_txt_data)
model = {}
for asp in asp_lst:
    model[asp] = MultinomialNB()
    x = [txt for txt, label in data[asp].items()]
    y = [label for txt, label in data[asp].items()]
    x = vectorizer.transform(x)
    model[asp].fit(x.toarray(), y)

#### Thử nghiệm trên tập val và test

In [ ]:
def predict(text, model, vectorizer, asp_lst):
    '''
    Lấy kết quả dự đoán từ model tương ứng với text, model là một dict các naive bayes tương ứng với từng aspect.
    '''
    result = []
    asp_map = {1: 'negative', 2: 'positive', 3: 'neutral'}
    for asp in asp_lst:
        pred = model[asp].predict([vectorizer.transform([text]).toarray()[0]])[0]
        if pred != 0:
            result.append((asp, asp_map[pred]))
    return result

In [ ]:
def check_pred(pred, truth):
    '''
    Trả về TP so giữa pred và truth.
    '''
    TP = len(list(set(pred) & set(truth)))
    return TP
def check_pred_asp(pred, truth):
    '''
    Trả về TP so giữa pred và truth, chỉ xét aspect.
    '''
    pred = [asp for asp, sent in pred]
    truth = [asp for asp, sent in truth]
    TP = len(list(set(pred) & set(truth)))
    return TP

In [ ]:
def F1(TP, pred, truth):
    '''
    Trả về chỉ số F1. 
    '''
    precision = TP / pred
    recall = TP / truth
    return 2 * precision * recall / (precision + recall)

In [ ]:
# dùng chỉ số F1 để đánh giá
val_data = preprocess_txt('/content/VLSP2018-SA-train-dev-test/2-VLSP2018-SA-Hotel-dev (7-3-2018).txt')
nTP, nTP_asp, npred, ntruth = 0, 0, 0, 0 
asp_f1 = {}
sent_f1 = {}
for pdata in val_data:
    pdata = (process_raw_dataset(pdata[0]), pdata[1])
    pred = predict(pdata[0], model, vectorizer, asp_lst)
    truth = pdata[1]

    nTP = nTP + check_pred(pred, truth)
    nTP_asp = nTP_asp + check_pred_asp(pred, truth)
    npred = npred + len(pred)
    ntruth = ntruth + len(truth)
    ints = set(pred) & set(truth)
    for m in ints:
        if m[0] not in asp_f1.keys():
            asp_f1[m[0]] = (0, 0, 0)
        if m[1] not in sent_f1.keys():
            sent_f1[m[1]] = (0, 0, 0)
        asp_f1[m[0]] = (asp_f1[m[0]][0] + 1, asp_f1[m[0]][1], asp_f1[m[0]][2])
        sent_f1[m[1]] = (sent_f1[m[1]][0] + 1, sent_f1[m[1]][1], sent_f1[m[1]][2])
    for m in set(pred) - ints:
        if m[0] not in asp_f1.keys():
            asp_f1[m[0]] = (0, 0, 0)
        if m[1] not in sent_f1.keys():
            sent_f1[m[1]] = (0, 0, 0)
        asp_f1[m[0]] = ((asp_f1[m[0]])[0], (asp_f1[m[0]])[1] + 1, (asp_f1[m[0]])[2])
        sent_f1[m[1]] = (sent_f1[m[1]][0], sent_f1[m[1]][1] + 1, sent_f1[m[1]][2])
    for m in set(truth) - ints:
        if m[0] not in asp_f1.keys():
            asp_f1[m[0]] = (0, 0, 0)
        if m[1] not in sent_f1.keys():
            sent_f1[m[1]] = (0, 0, 0)
        asp_f1[m[0]] = (asp_f1[m[0]][0], asp_f1[m[0]][1], asp_f1[m[0]][2] + 1)
        sent_f1[m[1]] = (sent_f1[m[1]][0], sent_f1[m[1]][1], sent_f1[m[1]][2] + 1)
print('TẬP TEST')
report = pd.DataFrame.from_dict(asp_f1, orient='index', columns=['TP', 'FP', 'FN'])
report['Precision'] = report['TP'] / (report['TP'] + report['FP'])
report['Recall'] = report['TP'] / (report['TP'] + report['FN'])
report['F1'] = 2 * (report['Precision'] * report['Recall']) / (report['Precision'] + report['Recall'])
report.drop(['TP', 'FN', 'FP'], axis=1, inplace=True)
print(report)
report = pd.DataFrame.from_dict(sent_f1, orient='index', columns=['TP', 'FP', 'FN'])
report['Precision'] = report['TP'] / (report['TP'] + report['FP'])
report['Recall'] = report['TP'] / (report['TP'] + report['FN'])
report['F1'] = 2 * (report['Precision'] * report['Recall']) / (report['Precision'] + report['Recall'])
report.drop(['TP', 'FN', 'FP'], axis=1, inplace=True)
print(report)
print('F1:', F1(nTP, npred, ntruth))

TẬP TEST
                                Precision    Recall        F1
LOCATION#GENERAL                 0.673163  0.791887  0.727715
SERVICE#GENERAL                  0.720645  0.852021  0.780846
ROOMS#DESIGN&FEATURES            0.378995  0.349474  0.363636
HOTEL#COMFORT                    0.414352  0.435523  0.424674
ROOM_AMENITIES#QUALITY           0.203540  0.188525  0.195745
ROOM_AMENITIES#CLEANLINESS       0.071429  0.009434  0.016667
HOTEL#DESIGN&FEATURES            0.308140  0.194853  0.238739
FACILITIES#DESIGN&FEATURES       0.270833  0.118182  0.164557
HOTEL#GENERAL                    0.655172  0.540284  0.592208
ROOM_AMENITIES#DESIGN&FEATURES   0.491979  0.220624  0.304636
FACILITIES#COMFORT               0.100000  0.014493  0.025316
FACILITIES#QUALITY               0.483871  0.148515  0.227273
FOOD&DRINKS#QUALITY              0.650633  0.607565  0.628362
ROOMS#CLEANLINESS                0.511905  0.595843  0.550694
ROOMS#COMFORT                    0.258278  0.137809  0.179724

In [ ]:
val_data = preprocess_txt('/content/VLSP2018-SA-train-dev-test/3-VLSP2018-SA-Hotel-test (8-3-2018).txt')
nTP, nTP_asp, npred, ntruth = 0, 0, 0, 0 
asp_f1 = {}
sent_f1 = {}
for pdata in val_data:
    pdata = (process_raw_dataset(pdata[0]), pdata[1])
    pred = predict(pdata[0], model, vectorizer, asp_lst)
    truth = pdata[1]

    nTP = nTP + check_pred(pred, truth)
    nTP_asp = nTP_asp + check_pred_asp(pred, truth)
    npred = npred + len(pred)
    ntruth = ntruth + len(truth)
    ints = set(pred) & set(truth)
    for m in ints:
        if m[0] not in asp_f1.keys():
            asp_f1[m[0]] = (0, 0, 0)
        if m[1] not in sent_f1.keys():
            sent_f1[m[1]] = (0, 0, 0)
        asp_f1[m[0]] = (asp_f1[m[0]][0] + 1, asp_f1[m[0]][1], asp_f1[m[0]][2])
        sent_f1[m[1]] = (sent_f1[m[1]][0] + 1, sent_f1[m[1]][1], sent_f1[m[1]][2])
    for m in set(pred) - ints:
        if m[0] not in asp_f1.keys():
            asp_f1[m[0]] = (0, 0, 0)
        if m[1] not in sent_f1.keys():
            sent_f1[m[1]] = (0, 0, 0)
        asp_f1[m[0]] = ((asp_f1[m[0]])[0], (asp_f1[m[0]])[1] + 1, (asp_f1[m[0]])[2])
        sent_f1[m[1]] = (sent_f1[m[1]][0], sent_f1[m[1]][1] + 1, sent_f1[m[1]][2])
    for m in set(truth) - ints:
        if m[0] not in asp_f1.keys():
            asp_f1[m[0]] = (0, 0, 0)
        if m[1] not in sent_f1.keys():
            sent_f1[m[1]] = (0, 0, 0)
        asp_f1[m[0]] = (asp_f1[m[0]][0], asp_f1[m[0]][1], asp_f1[m[0]][2] + 1)
        sent_f1[m[1]] = (sent_f1[m[1]][0], sent_f1[m[1]][1], sent_f1[m[1]][2] + 1)
print('TẬP TEST')
report = pd.DataFrame.from_dict(asp_f1, orient='index', columns=['TP', 'FP', 'FN'])
report['Precision'] = report['TP'] / (report['TP'] + report['FP'])
report['Recall'] = report['TP'] / (report['TP'] + report['FN'])
report['F1'] = 2 * (report['Precision'] * report['Recall']) / (report['Precision'] + report['Recall'])
report.drop(['TP', 'FN', 'FP'], axis=1, inplace=True)
print(report)
report = pd.DataFrame.from_dict(sent_f1, orient='index', columns=['TP', 'FP', 'FN'])
report['Precision'] = report['TP'] / (report['TP'] + report['FP'])
report['Recall'] = report['TP'] / (report['TP'] + report['FN'])
report['F1'] = 2 * (report['Precision'] * report['Recall']) / (report['Precision'] + report['Recall'])
report.drop(['TP', 'FN', 'FP'], axis=1, inplace=True)
print(report)
print('F1:', F1(nTP, npred, ntruth))

TẬP TEST
                                Precision    Recall        F1
SERVICE#GENERAL                  0.760776  0.848558  0.802273
ROOMS#CLEANLINESS                0.570707  0.565000  0.567839
ROOM_AMENITIES#CLEANLINESS       1.000000  0.065217  0.122449
LOCATION#GENERAL                 0.704846  0.723982  0.714286
ROOMS#COMFORT                    0.473684  0.193548  0.274809
ROOMS#GENERAL                    0.714286  0.087719  0.156250
ROOMS#DESIGN&FEATURES            0.360000  0.272727  0.310345
ROOM_AMENITIES#DESIGN&FEATURES   0.444444  0.194444  0.270531
ROOM_AMENITIES#COMFORT                NaN  0.000000       NaN
HOTEL#CLEANLINESS                0.444444  0.179104  0.255319
ROOM_AMENITIES#GENERAL           0.500000  0.031250  0.058824
ROOM_AMENITIES#QUALITY           0.235294  0.135593  0.172043
FOOD&DRINKS#STYLE&OPTIONS        0.595506  0.427419  0.497653
ROOMS#QUALITY                    0.000000  0.000000       NaN
FACILITIES#DESIGN&FEATURES       0.363636  0.123077  0.183908

### Restaurant

#### Huấn luyện mô hình

In [ ]:
processed_txt_data = preprocess_txt('/content/VLSP2018-SA-train-dev-test/1-VLSP2018-SA-Restaurant-train (7-3-2018).txt')
processed_txt_data = [(process_raw_dataset(text), label) for text, label in processed_txt_data]
data = process_label(processed_txt_data)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

In [ ]:
corpus = [text for txt in list(data.values()) for text in list(txt.keys())]
vectorizer = CountVectorizer()
vectorizer.fit_transform(corpus)

<34980x3822 sparse matrix of type '<class 'numpy.int64'>'
	with 1429968 stored elements in Compressed Sparse Row format>

In [ ]:
asp_lst = get_aspect_list(processed_txt_data)
model = {}
for asp in asp_lst:
    model[asp] = MultinomialNB()
    x = [txt for txt, label in data[asp].items()]
    y = [label for txt, label in data[asp].items()]
    x = vectorizer.transform(x)
    model[asp].fit(x.toarray(), y)

#### Thử nghiệm trên tập val và test

In [ ]:
def predict(text, model, vectorizer, asp_lst):
    '''
    Lấy kết quả dự đoán từ model tương ứng với text, model là một dict các naive bayes tương ứng với từng aspect.
    '''
    result = []
    asp_map = {1: 'negative', 2: 'positive', 3: 'neutral'}
    for asp in asp_lst:
        pred = model[asp].predict([vectorizer.transform([text]).toarray()[0]])[0]
        if pred != 0:
            result.append((asp, asp_map[pred]))
    return result

In [ ]:
def check_pred(pred, truth):
    '''
    Trả về TP so giữa pred và truth.
    '''
    TP = len(list(set(pred) & set(truth)))
    return TP
def check_pred_asp(pred, truth):
    '''
    Trả về TP so giữa pred và truth, chỉ xét aspect.
    '''
    pred = [asp for asp, sent in pred]
    truth = [asp for asp, sent in truth]
    TP = len(list(set(pred) & set(truth)))
    return TP

In [ ]:
def F1(TP, pred, truth):
    '''
    Trả về chỉ số F1. 
    '''
    precision = TP / pred
    recall = TP / truth
    return 2 * precision * recall / (precision + recall)

In [ ]:
val_data = preprocess_txt('/content/VLSP2018-SA-train-dev-test/2-VLSP2018-SA-Restaurant-dev (7-3-2018).txt')
nTP, nTP_asp, npred, ntruth = 0, 0, 0, 0 
asp_f1 = {}
sent_f1 = {}
for pdata in val_data:
    pdata = (process_raw_dataset(pdata[0]), pdata[1])
    pred = predict(pdata[0], model, vectorizer, asp_lst)
    truth = pdata[1]

    nTP = nTP + check_pred(pred, truth)
    nTP_asp = nTP_asp + check_pred_asp(pred, truth)
    npred = npred + len(pred)
    ntruth = ntruth + len(truth)
    ints = set(pred) & set(truth)
    for m in ints:
        if m[0] not in asp_f1.keys():
            asp_f1[m[0]] = (0, 0, 0)
        if m[1] not in sent_f1.keys():
            sent_f1[m[1]] = (0, 0, 0)
        asp_f1[m[0]] = (asp_f1[m[0]][0] + 1, asp_f1[m[0]][1], asp_f1[m[0]][2])
        sent_f1[m[1]] = (sent_f1[m[1]][0] + 1, sent_f1[m[1]][1], sent_f1[m[1]][2])
    for m in set(pred) - ints:
        if m[0] not in asp_f1.keys():
            asp_f1[m[0]] = (0, 0, 0)
        if m[1] not in sent_f1.keys():
            sent_f1[m[1]] = (0, 0, 0)
        asp_f1[m[0]] = ((asp_f1[m[0]])[0], (asp_f1[m[0]])[1] + 1, (asp_f1[m[0]])[2])
        sent_f1[m[1]] = (sent_f1[m[1]][0], sent_f1[m[1]][1] + 1, sent_f1[m[1]][2])
    for m in set(truth) - ints:
        if m[0] not in asp_f1.keys():
            asp_f1[m[0]] = (0, 0, 0)
        if m[1] not in sent_f1.keys():
            sent_f1[m[1]] = (0, 0, 0)
        asp_f1[m[0]] = (asp_f1[m[0]][0], asp_f1[m[0]][1], asp_f1[m[0]][2] + 1)
        sent_f1[m[1]] = (sent_f1[m[1]][0], sent_f1[m[1]][1], sent_f1[m[1]][2] + 1)
print('TẬP TEST')
report = pd.DataFrame.from_dict(asp_f1, orient='index', columns=['TP', 'FP', 'FN'])
report['Precision'] = report['TP'] / (report['TP'] + report['FP'])
report['Recall'] = report['TP'] / (report['TP'] + report['FN'])
report['F1'] = 2 * (report['Precision'] * report['Recall']) / (report['Precision'] + report['Recall'])
report.drop(['TP', 'FN', 'FP'], axis=1, inplace=True)
print(report)
report = pd.DataFrame.from_dict(sent_f1, orient='index', columns=['TP', 'FP', 'FN'])
report['Precision'] = report['TP'] / (report['TP'] + report['FP'])
report['Recall'] = report['TP'] / (report['TP'] + report['FN'])
report['F1'] = 2 * (report['Precision'] * report['Recall']) / (report['Precision'] + report['Recall'])
report.drop(['TP', 'FN', 'FP'], axis=1, inplace=True)
print(report)
print('F1:', F1(nTP, npred, ntruth))

TẬP TEST
                          Precision    Recall        F1
FOOD#QUALITY               0.803279  0.887069  0.843097
FOOD#PRICES                0.583756  0.317680  0.411449
FOOD#STYLE&OPTIONS         0.395277  0.746124  0.516779
RESTAURANT#GENERAL         0.301724  0.303030  0.302376
SERVICE#GENERAL            0.746914  0.530702  0.620513
AMBIENCE#GENERAL           0.593220  0.341463  0.433437
RESTAURANT#MISCELLANEOUS   0.000000  0.000000       NaN
DRINKS#PRICES              0.500000  0.022727  0.043478
LOCATION#GENERAL           0.500000  0.059701  0.106667
DRINKS#QUALITY             0.000000  0.000000       NaN
DRINKS#STYLE&OPTIONS       0.000000  0.000000       NaN
RESTAURANT#PRICES          0.000000  0.000000       NaN
          Precision    Recall        F1
positive   0.604606  0.739683  0.665358
neutral    0.500000  0.067919  0.119593
negative   0.428571  0.012987  0.025210
F1: 0.577636939791761


In [ ]:
val_data = preprocess_txt('/content/VLSP2018-SA-train-dev-test/3-VLSP2018-SA-Restaurant-test (8-3-2018).txt')
nTP, nTP_asp, npred, ntruth = 0, 0, 0, 0 
asp_f1 = {}
sent_f1 = {}
for pdata in val_data:
    pdata = (process_raw_dataset(pdata[0]), pdata[1])
    pred = predict(pdata[0], model, vectorizer, asp_lst)
    truth = pdata[1]

    nTP = nTP + check_pred(pred, truth)
    nTP_asp = nTP_asp + check_pred_asp(pred, truth)
    npred = npred + len(pred)
    ntruth = ntruth + len(truth)
    ints = set(pred) & set(truth)
    for m in ints:
        if m[0] not in asp_f1.keys():
            asp_f1[m[0]] = (0, 0, 0)
        if m[1] not in sent_f1.keys():
            sent_f1[m[1]] = (0, 0, 0)
        asp_f1[m[0]] = (asp_f1[m[0]][0] + 1, asp_f1[m[0]][1], asp_f1[m[0]][2])
        sent_f1[m[1]] = (sent_f1[m[1]][0] + 1, sent_f1[m[1]][1], sent_f1[m[1]][2])
    for m in set(pred) - ints:
        if m[0] not in asp_f1.keys():
            asp_f1[m[0]] = (0, 0, 0)
        if m[1] not in sent_f1.keys():
            sent_f1[m[1]] = (0, 0, 0)
        asp_f1[m[0]] = ((asp_f1[m[0]])[0], (asp_f1[m[0]])[1] + 1, (asp_f1[m[0]])[2])
        sent_f1[m[1]] = (sent_f1[m[1]][0], sent_f1[m[1]][1] + 1, sent_f1[m[1]][2])
    for m in set(truth) - ints:
        if m[0] not in asp_f1.keys():
            asp_f1[m[0]] = (0, 0, 0)
        if m[1] not in sent_f1.keys():
            sent_f1[m[1]] = (0, 0, 0)
        asp_f1[m[0]] = (asp_f1[m[0]][0], asp_f1[m[0]][1], asp_f1[m[0]][2] + 1)
        sent_f1[m[1]] = (sent_f1[m[1]][0], sent_f1[m[1]][1], sent_f1[m[1]][2] + 1)
print('TẬP TEST')
report = pd.DataFrame.from_dict(asp_f1, orient='index', columns=['TP', 'FP', 'FN'])
report['Precision'] = report['TP'] / (report['TP'] + report['FP'])
report['Recall'] = report['TP'] / (report['TP'] + report['FN'])
report['F1'] = 2 * (report['Precision'] * report['Recall']) / (report['Precision'] + report['Recall'])
report.drop(['TP', 'FN', 'FP'], axis=1, inplace=True)
print(report)
report = pd.DataFrame.from_dict(sent_f1, orient='index', columns=['TP', 'FP', 'FN'])
report['Precision'] = report['TP'] / (report['TP'] + report['FP'])
report['Recall'] = report['TP'] / (report['TP'] + report['FN'])
report['F1'] = 2 * (report['Precision'] * report['Recall']) / (report['Precision'] + report['Recall'])
report.drop(['TP', 'FN', 'FP'], axis=1, inplace=True)
print(report)
print('F1:', F1(nTP, npred, ntruth))

TẬP TEST
                          Precision    Recall        F1
FOOD#QUALITY               0.810865  0.881838  0.844864
FOOD#STYLE&OPTIONS         0.727723  0.729529  0.728625
DRINKS#STYLE&OPTIONS       1.000000  0.021739  0.042553
DRINKS#QUALITY             1.000000  0.014085  0.027778
RESTAURANT#GENERAL         0.554878  0.408072  0.470284
DRINKS#PRICES                   NaN  0.000000       NaN
RESTAURANT#MISCELLANEOUS        NaN  0.000000       NaN
LOCATION#GENERAL           1.000000  0.005587  0.011111
FOOD#PRICES                0.432432  0.145015  0.217195
AMBIENCE#GENERAL           0.855556  0.301961  0.446377
SERVICE#GENERAL            0.628205  0.280000  0.387352
RESTAURANT#PRICES          1.000000  0.013699  0.027027
          Precision    Recall        F1
positive   0.723795  0.566806  0.635753
neutral    0.475000  0.032149  0.060222
negative   1.000000  0.006289  0.012500
F1: 0.5128749668170959
